## Практическое задание

  1. Постройте нейронную сеть(берем простую линейную сеть, которую разбирали на уроке: меняем число слоев, число нейронов , типы активации, тип оптимизатора)  на датасет from sklearn.datasets import load_boston. 
  2. Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу  Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

  3. Поработайте с документацией TensorFlow 2. Найти 2-3 полезные команды TensorFlow, не разобранные на уроке (полезные для Вас).

In [1]:
import tensorflow as tf
from tensorflow import keras

from sklearn.datasets import load_boston
from sklearn.metrics import r2_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

In [2]:
boston = load_boston()
data = boston["data"]
data.shape

C:\Users\AVB\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.

(506, 13)

In [3]:
data = pd.DataFrame(data, columns=boston["feature_names"])
target = boston["target"]
data.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [4]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [5]:
scaler_data = MinMaxScaler()
train_data = scaler_data.fit_transform(data)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data, target, test_size=0.25)
X_train.shape

(379, 13)

###### 3 слоя

In [7]:
res_3_layers = pd.DataFrame(columns=['n_neurons', 'optimizer', 'metrics', 'res_metrics', 'loss', 'r2'])

opt = ['RMSprop', 'SGD', 'Adam']

for i in [50, 100, 200]:
    for j, i_optim in enumerate([keras.optimizers.RMSprop(),keras.optimizers.SGD(),keras.optimizers.Adam()]):
        for k_metric in ['MSE', 'MAE']:
      
            x_input = keras.layers.Input(shape=(13))
            x = keras.layers.Dense(i, activation='relu')(x_input)
            x_output = keras.layers.Dense(1)(x)
            model = keras.models.Model(x_input, x_output)

            model.compile(optimizer=i_optim,
                          loss='mae',
                          metrics=[k_metric])

            model.fit(X_train, y_train, epochs=15, batch_size=1, verbose=0)

            loss_metric = model.evaluate(X_train, y_train)

            y_pred = model.predict(X_test)
            r2 = r2_score(y_test, y_pred)

            res_3_layers = res_3_layers.append(pd.DataFrame({'n_neurons': [i],
                                           'optimizer': [opt[j]],
                                           'loss': [loss_metric[0]],
                                           'metrics': k_metric,
                                           'res_metrics': [loss_metric[1]],
                                           'r2': [r2]}),
                                            ignore_index=True)

12/12 [==============================] - 0s 1000us/step - loss: 3.2248 - MAE: 3.2248


In [8]:
res_3_layers.sort_values(by='r2', ascending=False)

,n_neurons,optimizer,metrics,res_metrics,loss,r2
3,50,SGD,MAE,2.563666,2.563666,0.770355
14,200,SGD,MSE,18.104923,2.900914,0.767247
15,200,SGD,MAE,2.652735,2.652735,0.765434
2,50,SGD,MSE,18.516134,2.988084,0.763028
17,200,Adam,MAE,3.224791,3.224791,0.751395
8,100,SGD,MSE,20.717827,2.772896,0.747111
9,100,SGD,MAE,2.844270,2.844270,0.737227
16,200,Adam,MSE,22.906948,2.982061,0.734791
11,100,Adam,MAE,3.099661,3.099661,0.723739
13,200,RMSprop,MAE,3.139171,3.139171,0.721168


Наилучший результат был показан при количестве нейронов = 50, optimizer = SGD, тип активации - relu и составил r2 = 0.770355

Добавим ещё 1 слой и посмотрим как это скажется на результате.

###### 4 слоя

In [9]:
res_4_layers = pd.DataFrame(columns=['n_neurons', 'optimizer', 'metrics', 'res_metrics', 'loss', 'r2'])

opt = ['RMSprop', 'SGD', 'Adam']

for i in [50, 100, 200]:
    for j, i_optim in enumerate([keras.optimizers.RMSprop(),keras.optimizers.SGD(),keras.optimizers.Adam()]):
        for k_metric in ['MSE', 'MAE']:
      
            x_input = keras.layers.Input(shape=(13))
            x = keras.layers.Dense(i, activation='relu')(x_input)
            x_1 = keras.layers.Dense(i, activation='relu')(x)
            x_output = keras.layers.Dense(1)(x)
            model = keras.models.Model(x_input, x_output)

            model.compile(optimizer=i_optim,
                          loss='mae',
                          metrics=[k_metric])

            model.fit(X_train, y_train, epochs=15, batch_size=1, verbose=0)

            loss_metric = model.evaluate(X_train, y_train)

            y_pred = model.predict(X_test)
            r2 = r2_score(y_test, y_pred)

            res_4_layers = res_4_layers.append(pd.DataFrame({'n_neurons': [i],
                                           'optimizer': [opt[j]],
                                           'loss': [loss_metric[0]],
                                           'metrics': k_metric,
                                           'res_metrics': [loss_metric[1]],
                                           'r2': [r2]}),
                                            ignore_index=True)

12/12 [==============================] - 0s 1ms/step - loss: 3.0182 - MAE: 3.0182


In [10]:
res_4_layers.sort_values(by='r2', ascending=False)

,n_neurons,optimizer,metrics,res_metrics,loss,r2
8,100,SGD,MSE,18.480621,2.655100,0.778423
9,100,SGD,MAE,2.575556,2.575556,0.768812
17,200,Adam,MAE,3.018175,3.018175,0.765211
2,50,SGD,MSE,19.110126,2.784371,0.763170
15,200,SGD,MAE,2.872953,2.872953,0.755969
14,200,SGD,MSE,19.895081,2.726727,0.752331
16,200,Adam,MSE,21.669762,3.065707,0.749822
13,200,RMSprop,MAE,3.120951,3.120951,0.729105
11,100,Adam,MAE,3.086847,3.086847,0.712691
10,100,Adam,MSE,25.319645,3.300207,0.712173


Наилучший результат был показан при количестве нейронов = 200, optimizer = SGD, тип активации - relu и составил r2 = 0.778423
Результат улучшился незначительно.

Точность нейросети помогли улучшить следующие факторы:
- выбор наиболее подходящего оптимизатора,
- подбор оптимального количества нейронов,
- случайный перезапуск сети (r2 колебался от 0,66 до 0,79 при прочих равных параметрах).

###### Поработайте с документацией TensorFlow 2. Найти 2-3 полезные команды TensorFlow, не разобранные на уроке (полезные для Вас).

###### 1. tf.image.resize

Позволяет изменять размер картинок до необходимых параметров. Очень полезная функция для работы с изображениями.

Пример синтексиса:

tf.image.resize(
    
                images, size, method=ResizeMethod.BILINEAR, preserve_aspect_ratio=False,
                antialias=False, name=None
    
                )

Дополнительный параметры:


- bilinear: Bilinear interpolation. If antialias is true, becomes a hat/tent filter function with radius 1 when downsampling.
- lanczos3: Lanczos kernel with radius 3. High-quality practical filter but may have some ringing, especially on synthetic images.
- lanczos5: Lanczos kernel with radius 5. Very-high-quality filter but may have stronger ringing.
- bicubic: Cubic interpolant of Keys. Equivalent to Catmull-Rom kernel. Reasonably good quality and faster than Lanczos3Kernel, particularly when upsampling.
- gaussian: Gaussian kernel with radius 3, sigma = 1.5 / 3.0.
- nearest: Nearest neighbor interpolation. antialias has no effect when used with nearest neighbor interpolation.
- area: Anti-aliased resampling with area interpolation. antialias has no effect when used with area interpolation; it always anti-aliases.
- mitchellcubic: Mitchell-Netravali Cubic non-interpolating filter. For synthetic images (especially those lacking proper prefiltering), less ringing than Keys cubic kernel but less sharp.


###### 2. tf.summary

Данный модуль предоставляет широкие возможности для визуализации и анализа данных.

Пример синтексиса:

writer = tf.summary.create_file_writer("/tmp/mylogs")

with writer.as_default():

  for step in range(100):
  
    # other model code would go here
    
    tf.summary.scalar("my_metric", 0.5, step=step)
    
    writer.flush()

Функции

- audio(...): Write an audio summary.

- create_file_writer(...): Creates a summary file writer for the given log directory.

- create_noop_writer(...): Returns a summary writer that does nothing.

- flush(...): Forces summary writer to send any buffered data to storage.

- graph(...): Writes a TensorFlow graph summary.

- histogram(...): Write a histogram summary.

- image(...): Write an image summary.

- record_if(...): Sets summary recording on or off per the provided boolean value.

- scalar(...): Write a scalar summary.

- should_record_summaries(...): Returns boolean Tensor which is True if summaries will be recorded.

- text(...): Write a text summary.

- trace_export(...): Stops and exports the active trace as a Summary and/or profile file.

- trace_off(...): Stops the current trace and discards any collected information.

- trace_on(...): Starts a trace to record computation graphs and profiling information.

- write(...): Writes a generic summary to the default SummaryWriter if one exists.